 **02 – DATA PREPROCESSING**

The purpose of this notebook is to clean and prepare the raw data so it’s suitable for analysis and modeling. This step includes loading the dataset, checking for issues like missing values or duplicates, and making sure the data types are correct. It also covers encoding categorical variables, scaling features, and preparing the train/test split.

**1. Load the Datasets**
   
This step loads all CSV files from the data/raw/ folder into a dictionary of pandas DataFrames, using the filenames (without .csv) as keys. The code handles potential encoding and mixed-type issues by specifying encoding (ISO-8859-1) and using dtype=str. This approach ensures easy access to and inspection of multiple datasets.

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import glob
import pandas as pd
import warnings
from pandas.errors import DtypeWarning

# Suppress DtypeWarning
warnings.simplefilter(action='ignore', category=DtypeWarning)

# Path to the raw data folder
raw_data_path = '/content/drive/MyDrive/yacht-data-insights/data/raw/'

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(raw_data_path, '*.csv'))

# Create a dictionary of DataFrames with the filename (without extension) as the key
dataframes = {}

for file in csv_files:
    name = os.path.splitext(os.path.basename(file))[0]  # filename without path or extension
    try:
        # Read the CSV with dtype=str to avoid mixed type issues
        df = pd.read_csv(file, encoding='ISO-8859-1', dtype=str, low_memory=False)
        dataframes[name] = df
        print(f"Loaded {name} with shape {df.shape}")
    except Exception as e:
        print(f"Error loading {name}: {e}")

# Optionally, check the first file to confirm
if csv_files:
    first_file = csv_files[0]
    preview_df = pd.read_csv(first_file, encoding='ISO-8859-1', dtype=str, low_memory=False)
    print(f"\nFirst file preview: {preview_df.head()}")
else:
    print("No CSV files found.")


Loaded named_anchorages_v1_20191205 with shape (166508, 10)
Loaded CVP_loitering_202411 with shape (684, 14)
Loaded named_anchorages_v1_20181108 with shape (119748, 7)
Loaded Weather-for-Boating-Activities with shape (1060, 6)
Loaded CVP_ports_202411 with shape (1410, 14)
Loaded boat_data with shape (9888, 10)
Loaded CVP_encounters_202411 with shape (348, 14)
Loaded sar_vessel_detections_pipev20231026_202410 with shape (268681, 10)
Loaded named_anchorages_v2_20201104 with shape (166515, 10)
Loaded boat_dataset with shape (10344, 38)
Loaded Boats_No_Price_dataset with shape (936, 26)
Loaded named_anchorages_v2_20221206 with shape (166482, 10)
Loaded sar_vessel_detections_pipev3_202411 with shape (248247, 10)
Loaded sar_vessel_detections_pipev3_202412 with shape (239081, 10)

First file preview:        s2id                  lat                 lon     label sublabel  \
0  3e4e429b   26.914042109356018   52.22031972443178   SHARJAH      NaN   
1  1a575de7  -7.7159917314685496   11.7245595

**2. Initial Checks: Data Types, Missing Values, and Duplicates**

This step involves checking the data types to ensure each column is correctly formatted. It also includes identifying any missing values by counting them for each column. Additionally, duplicate rows are checked for any repetitions. These checks help identify common data issues that need to be addressed before further analysis.

In [3]:
# Loop through each loaded DataFrame in the dictionary
for name, df in dataframes.items():
    print(f"\n🔍 Initial checks for: {name}")

    # Overview of the dataset
    print("\n📄 Dataset Info:")
    print(df.info())

    # Count missing values per column
    print("\n❓ Missing Values per Column:")
    print(df.isnull().sum())

    # Check for duplicates
    print("\n🔁 Number of Duplicate Rows:")
    print(df.duplicated().sum())



🔍 Initial checks for: named_anchorages_v1_20191205

📄 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166508 entries, 0 to 166507
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   s2id                   166508 non-null  object
 1   lat                    166508 non-null  object
 2   lon                    166508 non-null  object
 3   label                  166502 non-null  object
 4   sublabel               5586 non-null    object
 5   label_source           166508 non-null  object
 6   iso3                   166501 non-null  object
 7   distance_from_shore_m  166483 non-null  object
 8   drift_radius           166346 non-null  object
 9   at_dock                166507 non-null  object
dtypes: object(10)
memory usage: 12.7+ MB
None

❓ Missing Values per Column:
s2id                          0
lat                           0
lon                           0
label                  

**3. Merging Anchorages Datasets**

The anchorage datasets were merged into a single DataFrame to facilitate unified analysis. This step combined multiple datasets related to anchorages, ensuring that all relevant data from different timeframes and sources is present in one dataset. This is an essential step for streamlining further data analysis and avoiding redundant information across multiple datasets.

In [4]:
# 1. Merging the Anchorages datasets
# Assuming the datasets have already been loaded into the `dataframes` dictionary

# Select the anchorage datasets
anchorage_keys = [key for key in dataframes.keys() if 'anchorages' in key]

# Print the datasets being merged
print("Merging the following Anchorage datasets:")
for key in anchorage_keys:
    print(f"- {key}")

# Merge them into a single DataFrame
df_anchorages = pd.concat([dataframes[key] for key in anchorage_keys], ignore_index=True)

# Display quick check
print("\nCombined Anchorage dataset shape:", df_anchorages.shape)
print(df_anchorages.head())


Merging the following Anchorage datasets:
- named_anchorages_v1_20191205
- named_anchorages_v1_20181108
- named_anchorages_v2_20201104
- named_anchorages_v2_20221206

Combined Anchorage dataset shape: (619253, 12)
       s2id                  lat                 lon     label sublabel  \
0  3e4e429b   26.914042109356018   52.22031972443178   SHARJAH      NaN   
1  1a575de7  -7.7159917314685496   11.72455952408813  BLOCK 17      NaN   
2  3fcf5295    29.64207745041846   48.69670549586832  KAZ IRAQ      NaN   
3  3fcf52bf   29.644147649617455  48.701872570021656  KAZ IRAQ      NaN   
4  3fcf52bd   29.639744433742802  48.701768589783754  UMM QASR      NaN   

      label_source iso3 distance_from_shore_m         drift_radius at_dock  \
0  top_destination  IRN                 63000  0.05632213051303668   FALSE   
1  top_destination  AGO                134000   0.1111107365900863   FALSE   
2  top_destination  KWT                 33000  0.16258325996472844   FALSE   
3  top_destination  KWT

**4.Handling Missing Values Across Multiple Datasets**

This step ensures that all loaded datasets are properly cleaned by addressing missing values. A loop iterates through each dataset, checking for columns with missing data. For numerical columns such as 'Length' and 'Width', missing values are filled using the median value. For categorical columns like 'Category', missing values are filled with the mode (most frequent value). If the mode is not available, missing categorical data is replaced with 'Unknown'. This approach ensures that missing data is handled consistently across all datasets, preventing errors by verifying the existence of each column before making changes. After cleaning, the missing value count is rechecked to confirm successful handling.

In [5]:
import pandas as pd
import os

# Optional: enable future behavior to avoid downcasting warning
pd.set_option('future.no_silent_downcasting', True)

# Paths
raw_data_path = '/content/drive/MyDrive/yacht-data-insights/data/raw/'
cleaned_data_path = '/content/drive/MyDrive/yacht-data-insights/data/cleaned/'

# Ensure output folder exists
os.makedirs(cleaned_data_path, exist_ok=True)

# List CSV files
csv_files = [file for file in os.listdir(raw_data_path) if file.endswith('.csv')]

# Handle each dataset
for file in csv_files:
    file_path = os.path.join(raw_data_path, file)

    try:
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        # Report missing values before cleaning
        missing_before = df.isnull().sum()
        missing_before = missing_before[missing_before > 0]
        if not missing_before.empty:
            print(f"\nMissing values before cleaning in {file}:\n{missing_before}")
        else:
            print(f"\nNo missing values found in {file} before cleaning.")

        # Clean 'Length' and 'Width' if present and convert to numeric
        for col in ['Length', 'Width']:
            if col in df.columns:
                df[col] = df[col].astype(str).str.extract(r'([\d\.]+)', expand=False)
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # Fill missing values
        for column in df.columns:
            if df[column].isnull().sum() > 0:
                if pd.api.types.is_numeric_dtype(df[column]):
                    median_val = df[column].median()
                    df[column] = df[column].fillna(median_val)
                    print(f"Filled missing values in numeric column '{column}' with median: {median_val}")
                else:
                    mode_val = df[column].mode().dropna()
                    if not mode_val.empty:
                        df[column] = df[column].fillna(mode_val[0])
                        print(f"Filled missing values in categorical column '{column}' with mode: {mode_val[0]}")
                    else:
                        df[column] = df[column].fillna('Unknown')
                        print(f"Filled missing values in categorical column '{column}' with 'Unknown'")

        # Ensure object types are converted properly (after fillna)
        df = df.infer_objects(copy=False)

        # Report missing values after cleaning
        missing_after = df.isnull().sum()
        missing_after = missing_after[missing_after > 0]
        if not missing_after.empty:
            print(f"\nMissing values after cleaning in {file}:\n{missing_after}")
        else:
            print(f"\nNo missing values left in {file} after cleaning.")

        # Save cleaned file
        cleaned_file_path = os.path.join(cleaned_data_path, file)
        df.to_csv(cleaned_file_path, index=False)
        print(f"Cleaned: {file}")

    except Exception as e:
        print(f"Failed to process {file}: {e}")



Missing values before cleaning in named_anchorages_v1_20191205.csv:
label                         6
sublabel                 160922
iso3                          7
distance_from_shore_m        25
drift_radius                162
at_dock                       1
dtype: int64
Filled missing values in categorical column 'label' with mode: ZHOUSHAN
Filled missing values in categorical column 'sublabel' with mode: FUJAIRAH OFFSHORE
Filled missing values in categorical column 'iso3' with mode: CHN
Filled missing values in numeric column 'distance_from_shore_m' with median: 1000.0
Filled missing values in numeric column 'drift_radius' with median: 0.09844404256206604
Filled missing values in categorical column 'at_dock' with mode: False

No missing values left in named_anchorages_v1_20191205.csv after cleaning.
Cleaned: named_anchorages_v1_20191205.csv

No missing values found in CVP_loitering_202411.csv before cleaning.

No missing values left in CVP_loitering_202411.csv after cleaning.
Clean

**5. Removing Duplicates and Cleaning Data Types**

This step removes duplicate rows to ensure the dataset contains unique entries. Afterward, it verifies that each column has the correct data type, converting numerical columns to numeric types, categorical columns to categories, and date columns to datetime format. This ensures data consistency and prepares the dataset for accurate analysis and modeling.


In [6]:
# Handle duplicates
for file in csv_files:
    file_path = os.path.join(raw_data_path, file)

    try:
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        # Remove duplicates
        df = df.drop_duplicates()
        print(f"Removed duplicate rows in {file}. New row count: {len(df)}")

        # Check and convert data types after cleaning
        # Example for converting specific columns to the correct type
        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # Convert 'Date' column to datetime
        if 'Category' in df.columns:
            df['Category'] = df['Category'].astype('category')  # Convert 'Category' column to category type

        # Optionally, you can also convert numeric columns explicitly
        for col in ['Length', 'Width']:  # Example for numerical columns
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # Save the cleaned data after removing duplicates and adjusting data types
        cleaned_file_path = os.path.join(cleaned_data_path, file)
        df.to_csv(cleaned_file_path, index=False)
        print(f"Cleaned and processed {file} (duplicates removed and data types adjusted)")

    except Exception as e:
        print(f"Failed to process {file}: {e}")


Removed duplicate rows in named_anchorages_v1_20191205.csv. New row count: 166508
Cleaned and processed named_anchorages_v1_20191205.csv (duplicates removed and data types adjusted)
Removed duplicate rows in CVP_loitering_202411.csv. New row count: 684
Cleaned and processed CVP_loitering_202411.csv (duplicates removed and data types adjusted)
Removed duplicate rows in named_anchorages_v1_20181108.csv. New row count: 119748
Cleaned and processed named_anchorages_v1_20181108.csv (duplicates removed and data types adjusted)
Removed duplicate rows in Weather-for-Boating-Activities.csv. New row count: 125
Cleaned and processed Weather-for-Boating-Activities.csv (duplicates removed and data types adjusted)
Removed duplicate rows in CVP_ports_202411.csv. New row count: 1410
Cleaned and processed CVP_ports_202411.csv (duplicates removed and data types adjusted)
Removed duplicate rows in boat_data.csv. New row count: 9888
Cleaned and processed boat_data.csv (duplicates removed and data types ad

**6. Ensuring Target Availability and Encoding Categorical Variables**

After performing an initial cleaning check on the datasets by inspecting columns before and after cleaning, I noticed that some important target variables were missing—specifically, the Price column from the boat sales data and relevant maintenance data.

For maintenance, the original raw datasets (SAR vessel detections and loitering events) were reloaded. Rather than applying extensive cleaning, I selected a subset of relevant columns from each and merged them on vessel_id to create a combined maintenance-ready dataset.

For price prediction, since the Price column had been lost during initial cleaning, I reloaded the original boat dataset. I cleaned the Price column by converting it to numeric and filling missing values, and then applied one-hot encoding to categorical variables to prepare independent and dependent variables for modeling.

For route optimization, I standardized the latitude and longitude features and applied KMeans clustering. This produced cluster labels, which were added as a new feature to support further route analysis and optimization.

Finally, I integrated a weather dataset relevant to boating activities. After loading the file using the appropriate encoding to handle character issues, I selected core features such as wind speed, wave height, and overall weather conditions. Categorical variables such as Weather, Day of the Week, and Boat Technical Condition were encoded using label encoding, and the target variable (Decision) was also encoded to prepare the dataset for classification modeling. This completed the encoding phase and ensured all datasets were target-ready and suitable for the next steps in the machine learning workflow.

In [7]:
# Before cleaning
print("Columns before cleaning:")
print(df.columns)

# After cleaning
print("Columns after cleaning:")
print(df.columns)


Columns before cleaning:
Index(['scene_id', 'timestamp', 'lat', 'lon', 'presence_score', 'length_m',
       'mmsi', 'matching_score', 'fishing_score', 'matched_category'],
      dtype='object')
Columns after cleaning:
Index(['scene_id', 'timestamp', 'lat', 'lon', 'presence_score', 'length_m',
       'mmsi', 'matching_score', 'fishing_score', 'matched_category'],
      dtype='object')


In [3]:
import pandas as pd

# Load datasets
raw_data_path = '/content/drive/MyDrive/yacht-data-insights/data/raw/'

df_sar = pd.read_csv(f'{raw_data_path}sar_vessel_detections_pipev3_202412.csv', encoding='ISO-8859-1')
df_loiter = pd.read_csv(f'{raw_data_path}CVP_loitering_202411.csv', encoding='ISO-8859-1')

# Display columns for sanity check
print("SAR columns:", df_sar.columns)
print("Loitering columns:", df_loiter.columns)

# Select relevant columns
sar_cols_to_use = ['mmsi', 'presence_score', 'length_m', 'matching_score', 'fishing_score', 'matched_category']
loiter_cols_to_use = ['vessel_id', 'event_type', 'event_start', 'event_end', 'lat_mean', 'lon_mean']

df_sar_reduced = df_sar[sar_cols_to_use].copy()
df_loiter_reduced = df_loiter[loiter_cols_to_use].copy()

# Rename for clarity & consistency before merge
df_sar_reduced.rename(columns={'mmsi': 'vessel_id'}, inplace=True)

# Convert vessel_id columns to string type in both DataFrames to avoid merge errors
df_sar_reduced['vessel_id'] = df_sar_reduced['vessel_id'].astype(str)
df_loiter_reduced['vessel_id'] = df_loiter_reduced['vessel_id'].astype(str)

# Merge datasets on 'vessel_id' with outer join
df_maintenance_ready = pd.merge(df_sar_reduced, df_loiter_reduced, on='vessel_id', how='outer')

# Check the merged data
print("Merged dataset shape:", df_maintenance_ready.shape)
print(df_maintenance_ready.head())


SAR columns: Index(['scene_id', 'timestamp', 'lat', 'lon', 'presence_score', 'length_m',
       'mmsi', 'matching_score', 'fishing_score', 'matched_category'],
      dtype='object')
Loitering columns: Index(['event_id', 'event_type', 'vessel_id', 'event_start', 'event_end',
       'lat_mean', 'lon_mean', 'lat_min', 'lat_max', 'lon_min', 'lon_max',
       'event_info', 'event_vessels', 'event_geography'],
      dtype='object')
Merged dataset shape: (239765, 11)
  vessel_id  presence_score    length_m  matching_score  fishing_score  \
0       0.0        0.999211   28.513245        0.317074       0.804592   
1       0.0        0.998853   42.291348        0.226967       0.723040   
2       0.0        0.999838   19.994408        0.015303       0.941307   
3       0.0        0.983325   20.038578       10.793140       0.960768   
4       0.0        0.997541  168.448975        9.604926       0.012883   

  matched_category event_type event_start event_end  lat_mean  lon_mean  
0     noisy_vess

In [8]:
import pandas as pd

# Path to the raw data folder
raw_data_path = '/content/drive/MyDrive/yacht-data-insights/data/raw/'

# Reload the raw boat dataset with the proper encoding to avoid 'UnicodeDecodeError'
df_boat = pd.read_csv(f'{raw_data_path}boat_dataset.csv', encoding='ISO-8859-1')

# Check the columns to verify the dataset
print("Columns in boat dataset:", df_boat.columns)

# Clean and preprocess the 'Price' column if present
if 'Price' in df_boat.columns:
    # If Price column contains non-numeric values (e.g., 'N/A', '$', etc.), convert them to numeric
    df_boat['Price'] = pd.to_numeric(df_boat['Price'], errors='coerce')

    # Handle missing prices by filling with the median value
    df_boat['Price'] = df_boat['Price'].fillna(df_boat['Price'].median())

# Encoding categorical variables
# Check if any categorical columns need encoding
categorical_columns = df_boat.select_dtypes(include=['object']).columns

# Perform one-hot encoding on categorical variables
df_boat_encoded = pd.get_dummies(df_boat, columns=categorical_columns, drop_first=True)

# Define your independent variables (X) and dependent variable (y)
X = df_boat_encoded.drop(columns=['Price'])  # Drop Price if it's the dependent variable
y = df_boat_encoded['Price']  # Assuming Price is the target for prediction

# Check the shape of the final datasets
print("Shape of the independent variables (X):", X.shape)
print("Shape of the dependent variable (y):", y.shape)

# Now you can proceed with model training or further analysis


Columns in boat dataset: Index(['Price', 'Category', 'Boat Type', 'Manufacturer', 'Model', 'Boat name',
       'Type', 'Year Built', 'Condition', 'Length', 'Width', 'Depth',
       'Displacement', 'CE Design Category', 'Cert Number of People',
       'Number of Cabins', 'Number of beds', 'Hull Color', 'Number of Toilets',
       'Number of Bathrooms', 'Number of Showers', 'Material',
       'Fresh Water Cap', 'Holding Tank', 'Propulsion', 'Engine',
       'Engine Performance', 'Fuel Capacity', 'Fuel Type', 'Engine Hours',
       'Max Speed', 'Cruising Speed', 'Location', 'Advertisement Date',
       'Number of views last 7 days', 'Comments', 'Additional Comments',
       'Equipment'],
      dtype='object')
Shape of the independent variables (X): (10344, 40219)
Shape of the dependent variable (y): (10344,)


In [9]:
# Route Optimization (Clustering)
route_features = ['lat', 'lon']

if all(col in df.columns for col in route_features):
    X_route = df[route_features].dropna()

    from sklearn.preprocessing import StandardScaler
    X_route_scaled = StandardScaler().fit_transform(X_route)

    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=5, random_state=42)
    df['route_cluster'] = kmeans.fit_predict(X_route_scaled)

    print("🧭 Route Optimization")
    print(f"Route clusters assigned. Sample:\n{df[['lat', 'lon', 'route_cluster']].head()}")
else:
    print("Required GPS features not found.")


🧭 Route Optimization
Route clusters assigned. Sample:
         lat         lon  route_cluster
0  39.806985   25.831447              2
1  58.653740   21.553987              2
2   4.247573  119.633861              4
3   1.156218  119.212269              4
4   0.015330  117.588380              4


In [10]:
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Mount Google Drive
drive.mount('/content/drive')

# Path to the weather dataset
weather_path = '/content/drive/MyDrive/yacht-data-insights/data/raw/Weather-for-Boating-Activities.csv'

# Load the weather dataset with utf-8-sig encoding to handle BOM characters
df_weather = pd.read_csv(weather_path, encoding='utf-8-sig')

# Display columns for sanity check
print("Weather dataset columns:", df_weather.columns)

# Select relevant columns (adjust these as per your actual columns)
weather_cols_to_use = ['Wind Speed', 'Wave Height', 'Weather', 'Day of the Week', 'Boat Technical Condition', 'Decision']

df_weather_reduced = df_weather[weather_cols_to_use].copy()

# Check for missing values (optional)
print("Missing values per column:\n", df_weather_reduced.isnull().sum())

# Handle missing values if needed (example: fill with mode or drop)
df_weather_reduced = df_weather_reduced.dropna()

# Encode categorical variables
le_weather = LabelEncoder()
le_day = LabelEncoder()
le_boat_cond = LabelEncoder()
le_decision = LabelEncoder()

df_weather_reduced['Weather'] = le_weather.fit_transform(df_weather_reduced['Weather'])
df_weather_reduced['Day of the Week'] = le_day.fit_transform(df_weather_reduced['Day of the Week'])
df_weather_reduced['Boat Technical Condition'] = le_boat_cond.fit_transform(df_weather_reduced['Boat Technical Condition'])
df_weather_reduced['Decision'] = le_decision.fit_transform(df_weather_reduced['Decision'])

# Define independent variables X and dependent variable y
X = df_weather_reduced.drop('Decision', axis=1)
y = df_weather_reduced['Decision']

print("Encoded dataset preview:")
print(df_weather_reduced.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Weather dataset columns: Index(['Wind Speed', 'Wave Height', 'Weather', 'Day of the Week',
       'Boat Technical Condition', 'Decision'],
      dtype='object')
Missing values per column:
 Wind Speed                  0
Wave Height                 0
Weather                     0
Day of the Week             0
Boat Technical Condition    0
Decision                    0
dtype: int64
Encoded dataset preview:
  Wind Speed Wave Height  Weather  Day of the Week  Boat Technical Condition  \
0        Low         Low        0                0                         2   
1     Medium        High        2                0                         2   
2        Low        High        2                0                         2   
3     Medium         Low        2                0                         2   
4     Medium        High        2                1              

7. Scale Numerical Features
Numerical features were scaled using StandardScaler to ensure all features contribute equally to model performance.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Apply standard scaling
scaler = StandardScaler()
df[['numerical_column1', 'numerical_column2']] = scaler.fit_transform(df[['numerical_column1', 'numerical_column2']])

8. Split Dataset into Train/Test
The cleaned and prepared dataset was split into training and testing sets to allow model validation on unseen data.

In [ ]:
from sklearn.model_selection import train_test_split

# Separate features and target
X = df.drop('target_column', axis=1)
y = df['target_column']

# Create training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
